### Remove:
* Null values, new_line("\n"), hashtags("#*"), emojis, other characters
### Replace:
* ['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'] with ['ሰ, 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']
* ['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'] with ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']
* ['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'] with ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']


In [34]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from util import Util

In [37]:
parsed_dir = "../data/raw/"
cleaned_dir = "../data/parsed/"
file_name = "YeneTube"
util = Util()

Loading  the Raw files

In [38]:
df = pd.read_json(f"{parsed_dir}/{file_name}.json")
df.head()


,name,type,id,messages
0,YeneTube,public_channel,1105112305,"{'id': 1, 'type': 'service', 'date': '2017-04-..."
1,YeneTube,public_channel,1105112305,"{'id': 2, 'type': 'service', 'date': '2017-04-..."
2,YeneTube,public_channel,1105112305,"{'id': 403, 'type': 'message', 'date': '2017-0..."
3,YeneTube,public_channel,1105112305,"{'id': 423, 'type': 'message', 'date': '2017-0..."
4,YeneTube,public_channel,1105112305,"{'id': 424, 'type': 'message', 'date': '2017-0..."


In [39]:
df = pd.DataFrame(df['messages'])
df_messages = pd.DataFrame(df['messages'].tolist())
df_messages['text'] = df_messages['text'].apply(lambda x: ' '.join([i if isinstance(i, str) else i['text'] for i in x if isinstance(i, str) or ('text' in i and isinstance(i['text'], str))]))
df_messages = df_messages[['id', 'text', 'date']]
df_messages

,id,text,date
0,1,,2017-04-01T22:09:30
1,2,,2017-04-01T22:09:31
2,403,W E H I T 1 K 😁 😁 \n 🎆 🎇,2017-04-28T18:30:56
3,423,,2017-04-29T12:42:05
4,424,"Teddy Afro's 5th studio album, #Ethiopia wil...",2017-04-29T12:44:55
...,...,...,...
40328,48921,በደብረብርሃን ከተማ ወቅታዊ የጸጥታ ሁኔታን ምክኒያት በማድረግ የባጃጅ ተ...,2024-01-03T21:37:53
40329,48922,ማትረፍን በቃል ሳይሆን በተግባር አሳይቷል \n\nከሰባት በላይ ድርጅቶች ...,2024-01-04T10:51:55
40330,48923,ሜሎና ኢንተርቴመንት \n\nየድርጅትዎን ብራንድ ከlogo design ጀምሮ...,2024-01-04T11:02:38
40331,48924,የሻረግ ቅመማ-ቅመም 👌 \n #ምርቶቻችንን :- \n🎄 ከሸዋ ሾፒንግ \n...,2024-01-04T11:02:38


Remove null values

In [40]:
data_null_removed = df_messages.dropna()
data_null_removed.head()

,id,text,date
0,1,,2017-04-01T22:09:30
1,2,,2017-04-01T22:09:31
2,403,W E H I T 1 K 😁 😁 \n 🎆 🎇,2017-04-28T18:30:56
3,423,,2017-04-29T12:42:05
4,424,"Teddy Afro's 5th studio album, #Ethiopia wil...",2017-04-29T12:44:55


Remove new line

In [41]:
yenetube_data = data_null_removed.replace('\n', ' ', regex=True)
yenetube_data.head()

,id,text,date
0,1,,2017-04-01T22:09:30
1,2,,2017-04-01T22:09:31
2,403,W E H I T 1 K 😁 😁 🎆 🎇,2017-04-28T18:30:56
3,423,,2017-04-29T12:42:05
4,424,"Teddy Afro's 5th studio album, #Ethiopia wil...",2017-04-29T12:44:55


Extract and remove hasthags

In [42]:
# Extract hashtags
yenetube_data['hashtags'] = yenetube_data['text'].apply(lambda x: util.extract_hashtags(x))
yenetube_data.head()

,id,text,date,hashtags
0,1,,2017-04-01T22:09:30,[]
1,2,,2017-04-01T22:09:31,[]
2,403,W E H I T 1 K 😁 😁 🎆 🎇,2017-04-28T18:30:56,[]
3,423,,2017-04-29T12:42:05,[]
4,424,"Teddy Afro's 5th studio album, #Ethiopia wil...",2017-04-29T12:44:55,[#Ethiopia]


In [43]:
# Remove hashtags from text
yenetube_data['text'] = yenetube_data['text'].str.replace(r'\#\w+', '', regex=True)
yenetube_data.head()

,id,text,date,hashtags
0,1,,2017-04-01T22:09:30,[]
1,2,,2017-04-01T22:09:31,[]
2,403,W E H I T 1 K 😁 😁 🎆 🎇,2017-04-28T18:30:56,[]
3,423,,2017-04-29T12:42:05,[]
4,424,"Teddy Afro's 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia]


Extract emojies

In [44]:
# Extract emojis using regex
yenetube_data['emojis'] = yenetube_data['text'].apply(util.extract_emojis)
yenetube_data.tail()

,id,text,date,hashtags,emojis
40328,48921,በደብረብርሃን ከተማ ወቅታዊ የጸጥታ ሁኔታን ምክኒያት በማድረግ የባጃጅ ተ...,2024-01-03T21:37:53,[],
40329,48922,ማትረፍን በቃል ሳይሆን በተግባር አሳይቷል ከሰባት በላይ ድርጅቶች ከስ...,2024-01-04T10:51:55,[],
40330,48923,ሜሎና ኢንተርቴመንት የድርጅትዎን ብራንድ ከlogo design ጀምሮ እ...,2024-01-04T11:02:38,[],
40331,48924,የሻረግ ቅመማ-ቅመም 👌 :- 🎄 ከሸዋ ሾፒንግ 🎄 ከገበሬው(ፐርፐ...,2024-01-04T11:02:38,"[#ምርቶቻችንን, #ጥራት]",👌🎄🎄🎄🎄🎄🎄🎄🎄📲
40332,48925,በጋዛ በእስራኤል ጥቃት የተነሳ የአንድ ቤሰተብ አባል የሆኑ 14 ሰዎች ተ...,2024-01-04T11:08:01,[],


In [45]:
yenetube_data['text'] = yenetube_data['text'].apply(util.remove_emojis)
yenetube_data.head()

,id,text,date,hashtags,emojis
0,1,,2017-04-01T22:09:30,[],
1,2,,2017-04-01T22:09:31,[],
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇
3,423,,2017-04-29T12:42:05,[],
4,424,"Teddy Afro's 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],


#### Replace letters:
* ['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'] with ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']
* ['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'] with ['ሰ, 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']
* ['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'] with ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']
* ['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'] with ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']

In [46]:
letters = [
  [['ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሔ', 'ሖ'], ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']],
  [['ኀ', 'ኁ', 'ኂ', 'ኃ', 'ኄ', 'ኅ', 'ኆ'], ['ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ']],
  [['ሠ', 'ሡ', 'ሢ', 'ሣ', 'ሤ', 'ሦ', 'ሦ', 'ሧ'], ['ሰ', 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሷ']],
  [['ዐ', 'ዑ', 'ዒ', 'ዓ', 'ዔ', 'ዕ', 'ዖ'], ['አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ']],
  [['ጸ', 'ጹ', 'ጺ', 'ጻ', 'ጼ', 'ጽ', 'ጾ'], ['ፀ', 'ፁ', 'ፂ', 'ፃ', 'ፄ', 'ፅ', 'ፆ']]
]
for letter in letters:
  for i in range(len(letter[0])):
    yenetube_data['text'] = yenetube_data['text'].str.replace(letter[0][i], letter[1][i])

In [47]:
yenetube_data.head()

,id,text,date,hashtags,emojis
0,1,,2017-04-01T22:09:30,[],
1,2,,2017-04-01T22:09:31,[],
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇
3,423,,2017-04-29T12:42:05,[],
4,424,"Teddy Afro's 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],


Extract Symbols

In [48]:
yenetube_data['symbols'] = yenetube_data['text'].apply(util.extract_symbols)
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols
0,1,,2017-04-01T22:09:30,[],,
1,2,,2017-04-01T22:09:31,[],,
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,
3,423,,2017-04-29T12:42:05,[],,
4,424,"Teddy Afro's 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,'


In [49]:
yenetube_data['text'] = yenetube_data['text'].apply(util.remove_symbols)
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols
0,1,,2017-04-01T22:09:30,[],,
1,2,,2017-04-01T22:09:31,[],,
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,
3,423,,2017-04-29T12:42:05,[],,
4,424,"Teddy Afro s 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,'


Extract Links

In [50]:
yenetube_data['links'] = yenetube_data['text'].apply(util.extract_urls)
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols,links
0,1,,2017-04-01T22:09:30,[],,,[]
1,2,,2017-04-01T22:09:31,[],,,[]
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,,[]
3,423,,2017-04-29T12:42:05,[],,,[]
4,424,"Teddy Afro s 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,',[]


In [51]:
yenetube_data['links'] = yenetube_data['text'].apply(util.extract_urls)
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols,links
0,1,,2017-04-01T22:09:30,[],,,[]
1,2,,2017-04-01T22:09:31,[],,,[]
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,,[]
3,423,,2017-04-29T12:42:05,[],,,[]
4,424,"Teddy Afro s 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,',[]


Extract mentions

In [52]:
yenetube_data['mentions'] = yenetube_data['text'].apply(util.extract_mentions)
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols,links,mentions
0,1,,2017-04-01T22:09:30,[],,,[],[]
1,2,,2017-04-01T22:09:31,[],,,[],[]
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,,[],[]
3,423,,2017-04-29T12:42:05,[],,,[],[]
4,424,"Teddy Afro s 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,',[],[]


In [53]:
yenetube_data['text'] = yenetube_data['text'].str.replace(util.mention_pattern, '', regex=True).str.strip()
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols,links,mentions
0,1,,2017-04-01T22:09:30,[],,,[],[]
1,2,,2017-04-01T22:09:31,[],,,[],[]
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,,[],[]
3,423,,2017-04-29T12:42:05,[],,,[],[]
4,424,"Teddy Afro s 5th studio album, will be rele...",2017-04-29T12:44:55,[#Ethiopia],,',[],[]


Remove extra spaces 

In [54]:
yenetube_data['text'] = yenetube_data['text'].str.replace('\s+', ' ', regex=True).str.strip()


In [55]:
yenetube_data['text'] = yenetube_data['text'].replace(r'!+', '!', regex=True)
yenetube_data['text'] = yenetube_data['text'].replace(r'\.+', '', regex=True)

In [56]:
yenetube_data.head()

,id,text,date,hashtags,emojis,symbols,links,mentions
0,1,,2017-04-01T22:09:30,[],,,[],[]
1,2,,2017-04-01T22:09:31,[],,,[],[]
2,403,W E H I T 1 K,2017-04-28T18:30:56,[],😁😁🎆🎇,,[],[]
3,423,,2017-04-29T12:42:05,[],,,[],[]
4,424,"Teddy Afro s 5th studio album, will be release...",2017-04-29T12:44:55,[#Ethiopia],,',[],[]


save cleaned dataframe

In [57]:
yenetube_data = yenetube_data.drop(['hashtags', 'emojis', 'symbols', 'links', 'mentions'], axis=1)

In [58]:
yenetube_data

,id,text,date
0,1,,2017-04-01T22:09:30
1,2,,2017-04-01T22:09:31
2,403,W E H I T 1 K,2017-04-28T18:30:56
3,423,,2017-04-29T12:42:05
4,424,"Teddy Afro s 5th studio album, will be release...",2017-04-29T12:44:55
...,...,...,...
40328,48921,በደብረብርሃን ከተማ ወቅታዊ የፀጥታ ሁኔታን ምክኒያት በማድረግ የባጃጅ ተ...,2024-01-03T21:37:53
40329,48922,ማትረፍን በቃል ሳይሆን በተግባር አሳይቷል ከሰባት በላይ ድርጅቶች ከስሩ ...,2024-01-04T10:51:55
40330,48923,ሜሎና ኢንተርቴመንት የድርጅትዎን ብራንድ ከlogo design ጀምሮ እስከ...,2024-01-04T11:02:38
40331,48924,የሻረግ ቅመማ ቅመም : ከሸዋ ሾፒንግ ከገበሬው(ፐርፐዝብላክ) ከባምቢስ(ል...,2024-01-04T11:02:38


In [33]:
yenetube_data.to_csv(f"{cleaned_dir}/{file_name}.csv")